In [1]:
#Ejercicio: Transformaciones avanzadas en dataset de e-commerce

#Datos base:

import pandas as pd
import numpy as np

# Clientes
clientes = pd.DataFrame({
    'cliente_id': range(1, 6),
    'nombre': ['Ana', 'Juan', 'María', 'Pedro', 'Laura'],
    'segmento': ['Premium', 'Regular', 'Premium', 'Regular', 'VIP']
})

# Pedidos
pedidos = pd.DataFrame({
    'pedido_id': range(1, 11),
    'cliente_id': np.random.choice(range(1, 6), 10),
    'producto': np.random.choice(['A', 'B', 'C', 'D'], 10),
    'precio': np.random.uniform(50, 500, 10).round(2),
    'fecha': pd.date_range('2024-01-01', periods=10)
})

print("Clientes y pedidos cargados")

Clientes y pedidos cargados


In [2]:
#Enriquecer datos con joins:

# Unir pedidos con información de clientes
pedidos_enriquecidos = pd.merge(
    pedidos, 
    clientes, 
    on='cliente_id', 
    how='left'
)

print("Pedidos con información de clientes:")
print(pedidos_enriquecidos.head())

Pedidos con información de clientes:
   pedido_id  cliente_id producto  precio      fecha nombre segmento
0          1           1        A  221.90 2024-01-01    Ana  Premium
1          2           5        A  260.75 2024-01-02  Laura      VIP
2          3           3        B  172.69 2024-01-03  María  Premium
3          4           2        B  470.36 2024-01-04   Juan  Regular
4          5           1        A  222.07 2024-01-05    Ana  Premium


In [5]:
#Calcular métricas derivadas:

# Calcular métricas por cliente
metricas_cliente = pedidos_enriquecidos.groupby(['cliente_id', 'nombre', 'segmento']).agg({
    'pedido_id': 'count',
    'precio': ['sum', 'mean', 'max'],
    'fecha': 'max'  # Última compra
}).round(2)

# Aplanar columnas multi-nivel
metricas_cliente.columns = ['num_pedidos', 'total_gastado', 'gasto_promedio', 'gasto_maximo', 'ultima_compra']
metricas_cliente = metricas_cliente.reset_index()

print("\nMétricas por cliente:")
display(metricas_cliente)


Métricas por cliente:


,cliente_id,nombre,segmento,num_pedidos,total_gastado,gasto_promedio,gasto_maximo,ultima_compra
0,1,Ana,Premium,3,829.40,276.47,385.43,2024-01-07
1,2,Juan,Regular,2,680.06,340.03,470.36,2024-01-06
2,3,María,Premium,3,742.10,247.37,422.75,2024-01-09
3,4,Pedro,Regular,1,135.44,135.44,135.44,2024-01-10
4,5,Laura,VIP,1,260.75,260.75,260.75,2024-01-02


In [4]:
#Validar reglas de negocio:

def validar_reglas_negocio(df):
    validaciones = []
    
    # VIP deben tener al menos 2 pedidos
    vip_insuficientes = df[(df['segmento'] == 'VIP') & (df['num_pedidos'] < 2)]
    if len(vip_insuficientes) > 0:
        validaciones.append(f"VIPs con pocos pedidos: {len(vip_insuficientes)}")
    
    # Premium no deben exceder gasto máximo
    premium_excesivos = df[(df['segmento'] == 'Premium') & (df['gasto_maximo'] > 800)]
    if len(premium_excesivos) > 0:
        validaciones.append(f"Premiums con gastos excesivos: {len(premium_excesivos)}")
    
    return validaciones

reglas_incumplidas = validar_reglas_negocio(metricas_cliente)
print(f"\nReglas de negocio incumplidas: {reglas_incumplidas}")


Reglas de negocio incumplidas: ['VIPs con pocos pedidos: 1']
